In [2]:
import pandas as pd

from lxml import html
from bs4 import BeautifulSoup
import requests
import re

import time
import json

In [ ]:
import os
os.getcwd()

In [3]:
# 4장 - 분류에 맞는 데이터 분석 세트 준비 
# 설명변수가 여러개의 반응변수에 의해 설명이 되며, 반응변수들로 설명변수를 분류/추정/설명해 낼 수 있어야 한다.

page_lst = [1,2,3,4,5,6,7,8,9,10]
name_lst = []
gr_list = []
avgr_list = []
numv_list = []


In [4]:
# 1위 ~ 1000위의 bgg링크를 따오는 코드가 먼저 필요하다
# baselink로 메인 링크를 받고 몇 페이지까지 반복할 것인지에 대한 함수를 짜도록 하자
# html구조가 동적으로 생성됨 - xpath를 사용해야 한다
linklst = []
def getLinks(baseLink,tonum):
    
    for pn in range(1,tonum+1):
        url = baseLink+"/{}".format(pn)
        hdr = {'Accept-Language': 'ko_KR,en;q=0.8', 'User-Agent': ('Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Mobile Safari/537.36')}
        res = requests.get(url, headers=hdr)
        
        html = res.text
        soup = BeautifulSoup(html,'html.parser')
        
        bggTag = soup.find_all('a', class_='primary')
        
        for bg in bggTag:
            bggLink = bg['href']
            linklst.append(bggLink)
    return linklst

In [5]:
link_list = getLinks("https://boardgamegeek.com/browse/boardgame/page",10)

In [6]:
# getLinks로 가져온 링크들로 들어가서 필요한 정보들을 크롤링 한 후 리스트로 만들어서 반환하는 함수
# 셀레니움을 써야할듯 싶다. 왜 안나오는지 태그가 안 잡힌다.
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException


def getInfo(datlist):
    all_datlist = []
    driver = webdriver.Chrome()
    for link in datlist: #각 페이지에 일일히 들어가보게 된다.
        driver = webdriver.Chrome()
        url = f'https://boardgamegeek.com/{link}'
        driver.get(url)
        driver.implicitly_wait(2)
        
        bgg_rank = driver.find_element(By.XPATH, '//*[@id="mainbody"]/div[2]/div/div[1]/div[2]/ng-include/div/ng-include/div/div[2]/div[2]/div[1]/div/div[1]/overall-rating/div/div/a/span[1]').text
        name = driver.find_element(By.XPATH, '//*[@id="mainbody"]/div[2]/div/div[1]/div[2]/ng-include/div/ng-include/div/div[2]/div[2]/div[1]/div/div[2]/h1/a').text
        published_year = driver.find_element(By.XPATH, '//*[@id="mainbody"]/div[2]/div/div[1]/div[2]/ng-include/div/ng-include/div/div[2]/div[2]/div[1]/div/div[2]/h1/span').text
        plyr = driver.find_element(By.XPATH, '//*[@id="mainbody"]/div[2]/div/div[1]/div[2]/ng-include/div/ng-include/div/div[2]/div[2]/div[2]/gameplay-module/div/div/ul/li[1]/div[1]').text
        plyt = driver.find_element(By.XPATH, '//*[@id="mainbody"]/div[2]/div/div[1]/div[2]/ng-include/div/ng-include/div/div[2]/div[2]/div[2]/gameplay-module/div/div/ul/li[2]/div[1]').text
        recc_age = driver.find_element(By.XPATH, '//*[@id="mainbody"]/div[2]/div/div[1]/div[2]/ng-include/div/ng-include/div/div[2]/div[2]/div[2]/gameplay-module/div/div/ul/li[3]/div[1]').text
        weight = driver.find_element(By.XPATH, '//*[@id="mainbody"]/div[2]/div/div[1]/div[2]/ng-include/div/ng-include/div/div[2]/div[2]/div[2]/gameplay-module/div/div/ul/li[4]/div[1]/span[2]/span[1]').text
        game_type = driver.find_element(By.XPATH, '//*[@id="mainbody"]/div[2]/div/div[1]/div[2]/ng-include/div/div/ui-view/ui-view/div/overview-module/description-module/div/div[2]/div/div[1]/classifications-module/div/div[2]/ul/li[1]/div[2]').text
        
        mech_list = [2,3,4,5,6]
        mech_datlist = []
        
        for idx in mech_list:
            mech_path = f'//*[@id="mainbody"]/div[2]/div/div[1]/div[2]/ng-include/div/div/ui-view/ui-view/div/overview-module/description-module/div/div[2]/div/div[1]/classifications-module/div/div[2]/ul/li[3]/div[2]/popup-list/span[{idx}]/a'
            try:
                mech = driver.find_element(By.XPATH, mech_path).text
            except NoSuchElementException:
                mech = None
            
            mech_datlist.append(mech)
            
        
        
        datlist = [name, published_year , bgg_rank, plyr, plyt, recc_age, weight, game_type]
        datlist = datlist + mech_datlist
        
        all_datlist.append(datlist)
        driver = webdriver.Chrome()
        
    return all_datlist

In [ ]:
test = list(link_list[:3])
test

In [ ]:
test_dfs = getInfo(test)
test_dfs

In [9]:
# 더럽게 오래 걸리니까 100개씩 끊어서 만드는게 나을거 같다.
dfs_1 = getInfo(link_list[0:500])

In [15]:
dfs_2 = getInfo(link_list[100:200])

In [12]:
dfs_3 = getInfo(link_list[200:300])

In [15]:
dfs_4 = getInfo(link_list[300:400])

In [21]:
dfs_5 = getInfo(link_list[400:500])

In [ ]:
lest_dfs = getInfo(link_list[500:1001])

In [16]:
import pandas as pd
tmp = dfs_1 + lest_dfs
tmp = pd.DataFrame(tmp)

In [17]:
tmp.to_csv('bgg_1000_data.csv', index=True)

In [ ]:
dfs_5

In [14]:
lest_dfs = getInfo(link_list[500:1001])

In [ ]:
pd.DataFrame(tmp)